In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from ad import adnumber, jacobian

numeric parameters

In [3]:
dynamics = {
        'mass': 1.,
        'moment_of_inertia': 1.,
        'friction': .5,
        'gravity': 10.,
        'sampling_time': .1,
}

joint_limits = {
        'b': {'min': np.array([[-.2],[-.1],[-.1]]), 'max': np.array([[.2],[.1],[.1]])}, # x,y,theta
        'lf': {'min': np.array([[.0],[-.7]]), 'max': np.array([[.4],[-.3]])}, # wrt body
        'rf': {'min': np.array([[-.4],[-.7]]), 'max': np.array([[0.],[-.3]])}, # wrt body
        'lh': {'min': np.array([[.2],[-.2]]), 'max': np.array([[.4],[.2]])}, # wrt body
        'rh': {'min': np.array([[-.4],[-.2]]), 'max': np.array([[-.2],[.2]])}, # wrt body
        }

velocity_limits = {
        'b': {'min': np.array([[-2.],[-2.],[-.5]]), 'max': np.array([[2.],[2.],[.5]])}, # x,y,theta
        'lf': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        'rf': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        'lh': {'min': -2*np.ones((2,1)), 'max': 2*np.ones((2,1))},
        'rh': {'min': -2*np.ones((2,1)), 'max': 2*np.ones((2,1))},
        }

weight = dynamics['mass'] * dynamics['gravity']

controller ={
        'horizon': 10,
        'objective_norm': 'two',
        'state_cost': {
                'qb': 1.,
                'tb': 1.,
                'vb': 1.,
                'ob': 1.,
                'qlf_rel': 1.,
                'qrf_rel': 1.,
                'qlh_rel': .2,
                'qrh_rel': .2,
                },
        'input_cost': {
                'vlf': 1.,
                'vrf': 1.,
                'vlh': .2,
                'vrh': .2,
                },
        'gurobi': {
                'OutputFlag': 1,
                'TimeLimit': 1800.,
                'MIPFocus': 0,           # balanced: 0, feasibility: 1, optimality: 2, bounds: 3
                'NumericFocus': 0,       # min:     0, def:     0, max:     3
                'OptimalityTol': 1.e-6,  # min: 1.e-9, def: 1.e-6, max: 1.e-2 
                'FeasibilityTol': 1.e-6, # min: 1.e-9, def: 1.e-6, max: 1.e-2 
                'IntFeasTol': 1.e-5,     # min: 1.e-9, def: 1.e-5, max: 1.e-1 
                'MIPGap': 1.e-4,         # min:    0., def: 1.e-4, max:   inf
        }
}

visualizer = {
        'min': np.array([[-.6], [-.6]]),
        'max': np.array([[.6], [.4]]),
        'depth': [-.3,.3],
        'box_fixed_feet': {'tickness': .05, 'width': .1},
        'body_size': .2,
        'limbs_size': .05,
        'body_color': np.hstack((np.array([0.,0.,1.]), 1.)),
        'limbs_color': np.hstack((np.array([1.,0.,0.]), 1.)),
}

limbs parameters

In [4]:
'''
Domains of the limbs are polyhedrons in the form {q | A q <= b}.
The contact surface coincides with the first inequality, i.e. A_0 q = b_0.
'''

def R(th):
    return np.array([[np.cos(th), -np.sin(th)],[np.sin(th), np.cos(th)]])

class Limb():
    def __init__(self, A, b, q_nom):
        self.A = A
        self.b = b
        self.q_nom = q_nom
        self.n = - A[0,:].reshape(A.shape[1], 1)
        self.d = - b[0,0]
        self.t = R(np.pi/2.).dot(self.n)
        self.R = np.hstack((self.t, self.n))

# right hand
A_rh = np.array([[-1., 0.],[0., -1.]])
b_rh = np.array([[.5],[.5]])
q_rh = np.array([[-.3], [.0]])
rh = Limb(A_rh, b_rh, q_rh)

# left foot
A_lf = np.array([[0., -1.],[-1., 0.]])
b_lf = np.array([[.5],[.5]])
q_lf = np.array([[.2], [-.5]])
lf = Limb(A_lf, b_lf, q_lf)

# right foot
A_rf = np.array([[0., -1.],[-1., 0.]])
b_rf = np.array([[.5],[.5]])
q_rf = np.array([[-.2], [-.5]])
rf = Limb(A_rf, b_rf, q_rf)

# list of limbs
limbs = [rh, lf, rf]

symbolic state and inputs

In [5]:
def cross(u, v):
    return u[0,0]*v[1,0] - u[1,0]*v[0,0]

# body position and velocity
pb = adnumber(np.zeros((2,1)))
vb = adnumber(np.zeros((2,1)))
th = adnumber(np.zeros((1,1)))
om = adnumber(np.zeros((1,1)))

# limb positions
pl = []
for i in range(len(limbs)):
    pl.append(adnumber(np.zeros((2,1))))

# inputs
vl = []
for i in range(len(limbs)):
    vl.append(adnumber(np.zeros((2,1))))

# contact forces
f = []
for i in range(len(limbs)):
    f.append(adnumber(np.zeros((2,1))))

# state
state = np.vstack([pb, th] + pl + [vb, om])
inputs = np.vstack(vl + f)
    
# gravity
g = np.array([[0.], [-dynamics['gravity']]])

# other parameters
m = dynamics['mass']
I = dynamics['moment_of_inertia']
h = dynamics['sampling_time']
mu = dynamics['friction']

In [6]:
# body position and velocity
vb_next = vb + h*g
for i, l in enumerate(limbs):
    vb_next = vb_next + h/m*l.R.dot(f[i])
pb_next = pb + h*vb_next
om_next = om
for i, l in enumerate(limbs):
    om_next = om_next + h/I*cross(l.q_nom, l.R.dot(f[i]))
th_next = th + h*om_next

# limb positions
pl_next = [pl[i] + h*vl[i] for i, l in enumerate(limbs)]

In [7]:
state_next = np.vstack([pb_next, th_next] + pl_next + [vb_next, om_next])

A = np.array(jacobian(state_next.flatten().tolist(), state.flatten().tolist()))
B = np.array(jacobian(state_next.flatten().tolist(), inputs.flatten().tolist()))
c = np.array([[el[0].x] for el in state_next])

In [10]:
active_contacts = [0, 1]
inactive_contacts = [i for i in range(len(limbs)) if i not in active_contacts]

equalities = []
inequalities = []

# update for active limbs 
pl_active_next = []
for i in active_contacts:
    l = limbs[i]
    pl_active_next.append(l.d*l.n + l.t*(l.t.T*pl[i]))